In [41]:
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras import optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

<Strong> Importing a pre-built network (convolutional layer) </Strong>

In [20]:
from tensorflow.keras.applications import VGG16
conv_base = VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(150, 150, 3))

In [21]:
conv_base.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 150, 150, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 150, 150, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 75, 75, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 75, 75, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 75, 75, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 37, 37, 128)       0     

<Strong> 1. Feature Extraction - Reading Convolutional base over dataset then storing results in an array. </Strong>

In [22]:
import os
import numpy as np
from keras.preprocessing.image import ImageDataGenerator

In [23]:
base_dir = '/Users/jackyboy/Downloads/cats_and_dogs_small' 
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation') 
test_dir = os.path.join(base_dir, 'test')

<Strong> ImageDataGenerator lets you augment your images during training time. </Strong>

In [90]:
datagen = ImageDataGenerator(rescale=1./255)

train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

<Strong> Extracting features by calling predict on conv base model. </Strong>

In [98]:
generator = train_datagen.flow_from_directory(
        train_dir, 
        target_size=(150, 150),
        batch_size=20, 
        class_mode='binary',
        shuffle = False)

genny = datagen.flow_from_directory(
        train_dir, 
        target_size=(150, 150),
        batch_size=20, 
        class_mode='binary',
        shuffle = False) 

gennay = datagen.flow_from_directory(
        train_dir, 
        target_size=(150, 150),
        batch_size=20, 
        class_mode='binary',
        shuffle = False) 

Found 2000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [99]:
print(gennay[0])

print("*********************")

print(generator[0])

(array([[[[0.7960785 , 0.6431373 , 0.34117648],
         [0.8196079 , 0.6666667 , 0.3647059 ],
         [0.8196079 , 0.6666667 , 0.3647059 ],
         ...,
         [0.9686275 , 0.8078432 , 0.48627454],
         [0.9568628 , 0.8000001 , 0.4666667 ],
         [0.94117653, 0.78823537, 0.4784314 ]],

        [[0.7960785 , 0.6431373 , 0.34117648],
         [0.8196079 , 0.6666667 , 0.3647059 ],
         [0.8196079 , 0.6666667 , 0.3647059 ],
         ...,
         [0.9607844 , 0.8117648 , 0.48627454],
         [0.9607844 , 0.8000001 , 0.4784314 ],
         [0.94117653, 0.78823537, 0.4784314 ]],

        [[0.7960785 , 0.6431373 , 0.34117648],
         [0.8196079 , 0.6666667 , 0.3647059 ],
         [0.8196079 , 0.6666667 , 0.3647059 ],
         ...,
         [0.9686275 , 0.8196079 , 0.5019608 ],
         [0.9568628 , 0.8078432 , 0.4901961 ],
         [0.9490197 , 0.7960785 , 0.48627454]],

        ...,

        [[0.61960787, 0.48627454, 0.20784315],
         [0.62352943, 0.4901961 , 0.21176472

<Strong> Uses the same 2000 images in training for every epoch</Strong>

In [100]:
def extract_features(directory, sample_count):
    
    features = np.zeros(shape=(sample_count, 4, 4, 512)) 
    labels = np.zeros(shape=(sample_count))
    
    generator = datagen.flow_from_directory(
        directory, 
        target_size=(150, 150),
        batch_size=batch_size, 
        class_mode='binary')  
    
    i=0
    # 20 , 150 , 150 , 3  -  20 - for each batch label
    for inputs_batch, labels_batch in generator:

        features_batch = conv_base.predict(inputs_batch)
        features[i * batch_size : (i + 1) * batch_size] = features_batch 
        labels[i * batch_size : (i + 1) * batch_size] = labels_batch
        i += 1
    
        if i * batch_size >= sample_count:
            break
            
    return features, labels


In [57]:
train_features, train_labels = extract_features(train_dir, 2000) 
validation_features, validation_labels = extract_features(validation_dir, 1000) 
test_features, test_labels = extract_features(test_dir, 1000)

Found 2000 images belonging to 2 classes.
100


KeyboardInterrupt: 

<Strong> Before giving to classifier we need to flatten </Strong>

In [28]:
train_features = np.reshape(train_features, (2000, 4 * 4 * 512))
validation_features = np.reshape(validation_features, (1000, 4 * 4 * 512))
test_features = np.reshape(test_features, (1000, 4 * 4 * 512))

<Strong> Defining Classifier , with dropout for oversubscription </Strong>

<Strong> Training is very fast, because you only have to deal with two Dense layers.<Strong>

In [29]:
from keras import models
from keras import layers
from tensorflow.keras import optimizers

model = models.Sequential()
model.add(layers.Dense(256, activation='relu', input_dim=4 * 4 * 512))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer=optimizers.RMSprop(learning_rate=2e-5),
              loss='binary_crossentropy',
              metrics=['acc'])


In [ ]:
history = model.fit(train_features, train_labels,
                    epochs=30,
                    batch_size=20,
                    validation_data=(validation_features, validation_labels))

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

<Strong> 2. Overfitting after about 5 epoch , lets try data augmentation by extending current conv network. </Strong>

<Strong> Models act like layers , so we can add conv to Sequential model , just like a layer </Strong>

In [31]:
from keras import models
from keras import layers

model = models.Sequential()

model.add(conv_base)

model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [32]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 4, 4, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 8192)              0         
                                                                 
 dense_6 (Dense)             (None, 256)               2097408   
                                                                 
 dense_7 (Dense)             (None, 1)                 257       
                                                                 
Total params: 16,812,353
Trainable params: 16,812,353
Non-trainable params: 0
_________________________________________________________________


<Strong> We have to freeze the weights being updated on convolutional layer. </Strong>

In [33]:
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers

conv_base.trainable = False

In [34]:
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers

In [35]:
train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

In [36]:
test_datagen = ImageDataGenerator(rescale=1./255)

In [37]:
train_generator = train_datagen.flow_from_directory(
train_dir,
target_size=(150, 150),
batch_size=20,
class_mode='binary')

Found 2000 images belonging to 2 classes.


In [38]:
validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=(150, 150),
        batch_size=20,
        class_mode='binary')

Found 1000 images belonging to 2 classes.


In [42]:
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(learning_rate=2e-5),
              metrics=['acc'])

<Strong> Uses different set of 2000 images for every epoch.  </Strong>

In [43]:
history = model.fit_generator(
      train_generator,
      steps_per_epoch=100,
      epochs=30,
      validation_data=validation_generator,
      validation_steps=50)

Epoch 1/30


/var/folders/_f/jpr9q2w52wzg1v439ryj1bk00000gn/T/ipykernel_82491/3259228942.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(


 92/100 [==========================>...] - ETA: 8s - loss: 0.5730 - acc: 0.7060

KeyboardInterrupt: 

In [101]:
conv_base.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 150, 150, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 150, 150, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 75, 75, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 75, 75, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 75, 75, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 37, 37, 128)       0     

<Strong> As we have a small data-set we know we are at risk of overfitting , so we are only fine tuning the top layers of our Conv Base. </Strong>

<Strong> We are freezing upto a specific base in conv layer. </Strong>

In [102]:
conv_base.trainable = True
set_trainable = False
for layer in conv_base.layers:
    if layer.name == 'block5_conv1':
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

In [104]:
model.compile(loss='binary_crossentropy',
                    optimizer=optimizers.RMSprop(learning_rate=1e-5),
                    metrics=['acc'])

In [105]:
history = model.fit_generator(
              train_generator,
              steps_per_epoch=100,
              epochs=100,
              validation_data=validation_generator,
              validation_steps=50)

Epoch 1/100


/var/folders/_f/jpr9q2w52wzg1v439ryj1bk00000gn/T/ipykernel_82491/1168948864.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(


 30/100 [========>.....................] - ETA: 1:29 - loss: 0.4822 - acc: 0.7717

KeyboardInterrupt: 